Работу выполняли:

Бригинский Макар - БЭК2110

Кузнецова Милена -- БЭК2111

Михайлина Валерия -- БЭК2111

## 1. Парсинг

Импортируем нужные библиотеки:

In [26]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import random
from collections import defaultdict

Шаг 2: 4-5 числовых признаков и несколько категориальных

Шаг 3: Предварительныя обработка (максимум 10 баллов)
Проанализируйте столбцы. Что в них содержится? Опишите переменные. Есть ли в данных пропуски? Обработайте пропуски. Если в какой-то колонке пропусков большинство, то ее можно удалить. Если их не очень много, можно заполнить их медианой/средним или чем-нибудь еще, напишите почему вы выбрали тот или иной вариант. 

Шаг 4. Визуализация (максимум 20 баллов)
Сделайте для своих данных разведочный анализ данных(EDA). Выясните, какие в данных есть особенности, визуализируйте их. Ваша задача — откопать в данных интересные закономерности и построить для них красивые, понятные визуализации, на основе которых можно выдвинуть гипотезы и создать для датасета новые переменные.

Мы будем парсить Cуточно.Ру, а именно варианты посуточного проживания в Москве. 
Напишем функцию, которая будет собирать необходимые данные с сайта:

In [27]:
import requests
from bs4 import BeautifulSoup
from IPython.display import display, Markdown

# Задаем начальную страницу и количество страниц, которые нужно спарсить
start_page = 1
num_pages = 10

# Используем цикл, чтобы спарсить все страницы
for page in range(start_page, start_page + num_pages):
    url = f"https://www.sutochno.ru/?page={page}&from=mainpage"
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    cards = soup.find_all('div', class_='card')

    for i, card in enumerate(cards, start=1):
        name_of_hotel = card.find("a", class_="card-content__object-type js-name-click").text
        price = card.find("div", class_="price js-popover popover-wrap").find_all("span")[1].text.replace('₽', '')
        bedrooms = card.find_all("p", class_="facilities__facility tmp-font--medium")
        if len(bedrooms) > 2:
            bedrooms = bedrooms[2].text.strip().replace('спальня', '').replace('спальни', '')
        else:
            bedrooms = "-"
        area = card.find("span", class_="top-by-object-link js-popover popover-wrap").find("span", class_="value").text.replace('м', '')
        url = card.find("a", class_="obj-link").get("href")
        subway_element = card.find("span", class_="metro-title text-ellipsis")
        if subway_element:
            subway = subway_element.text.split('~')[0].strip()
            subway_time = subway_element.text.split('~')[1].split(' ')[1].strip().replace('мин', '')
        else:
            subway = "не указано"
            subway_time = "не указано"
        rating_element = card.select_one('.rating-top, .rating-list__rating')
        if rating_element:
            rating = rating_element.text.split(',')[0]
        else:
            rating = "не указано"

        display(Markdown(f"### {(page - 1) * len(cards) + i}. {name_of_hotel}"))
        display(Markdown(f"**Цена:** {price}"))
        display(Markdown(f"**Количество спален:** {bedrooms}"))
        display(Markdown(f"**Площадь:** {area}"))
        display(Markdown(f"**Ближайшее метро:** {subway}"))
        display(Markdown(f"**Время до ближайшего метро:** {subway_time}"))
        display(Markdown(f"**Рейтинг:** {rating}"))

### 1. SPA VIP апартаменты с сауной, джакузи в центре Москвы

**Цена:** 8 650 

**Количество спален:** 2 

**Площадь:** 85 

**Ближайшее метро:** ЗИЛ

...

**Количество спален:** 2 

**Площадь:** 70 

**Ближайшее метро:** Фрунзенская

**Время до ближайшего метро:** 3

**Рейтинг:** 8

Тут у нас получились повторы в объявлениях, о которых мы писали в форме. 

Поэтому мы решили парсить другой сайт, мы не очень много успели, так как пытались разобраться с Суточно.Ру. 
Вот что пока есть: 

In [28]:
import requests
from bs4 import BeautifulSoup

start_url = "https://m2.ru/moskva/nedvizhimost/snyat-kvartiru/s-balkonom/?pageNumber="
num_pages = 1
urls = []

for page in range(1, num_pages + 1):
    url = start_url + str(page)
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    cards = soup.find_all('div', class_='ClOfferSnippet__gallery')
    
    for card in cards:
        urls.append(card.find('a').get('href'))
        
for url in urls:
    print(url)

https://m2.ru/moskva/nedvizhimost/arenda-3-komnatnoi-kvartiri-83-9-m-8-17-etazh-12c8fe1c-3be3-48f8-bda4-619449095388/
https://m2.ru/moskva/nedvizhimost/arenda-2-komnatnoi-kvartiri-59-m-11-17-etazh-3509dde7-d342-46fe-920f-1bfa884d5b10/
https://m2.ru/moskva/nedvizhimost/arenda-2-komnatnoi-kvartiri-47-2-m-8-14-etazh-a1e3c731-2934-49c6-872d-3babebddff6f/
https://m2.ru/moskva/nedvizhimost/arenda-2-komnatnoi-kvartiri-50-m-4-9-etazh-cb6451e3-5102-46de-a965-01b5dfc5c599/
https://m2.ru/moskva/nedvizhimost/arenda-1-komnatnoi-kvartiri-32-m-8-12-etazh-c8cbecad-5a16-4784-bf8b-7a0ea3222c9f/
https://m2.ru/moskva/nedvizhimost/arenda-1-komnatnoi-kvartiri-37-m-13-16-etazh-39df6b6a-b271-4f19-9cc2-fac39a59a1fa/
https://m2.ru/moskva/nedvizhimost/arenda-3-komnatnoi-kvartiri-65-m-6-9-etazh-2ce27382-d5e5-46bd-9165-640507c32a18/
https://m2.ru/moskva/nedvizhimost/arenda-4-komnatnoi-kvartiri-162-m-14-17-etazh-2dcd825b-d10d-4627-9dd9-700b036ddb8e/
https://m2.ru/moskva/nedvizhimost/arenda-3-komnatnoi-kvartiri-92-m

Тут у нас выводятся первые 20 ссылок на объявления. Если изменить параметр num_pages, то тогда выведется больше. 
Чтобы не засорять ноутбок, поставили параметр 1. 